In [1]:
%matplotlib inline
import json
import codecs
from __future__ import print_function
import matplotlib.pyplot as plt
import os
import numpy as np
import random
from scipy import io as spio
import numpy
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils
from keras import backend as K
from keras.callbacks import *
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
from scipy.spatial.distance import cosine
import tensorflow as tf
K.set_image_dim_ordering('th')

/Users/asimonoff/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# Data Loading and Shaping

In [2]:
random.seed(2018) #setting a random seed so work is reproducible

In [3]:
emnist = spio.loadmat("../data/EMNIST/emnist-digits.mat")

In [4]:
#Loading the training and test sets
x_train = emnist["dataset"][0][0][0][0][0][0].astype(np.float32)
y_train = emnist["dataset"][0][0][0][0][0][1]

x_test = emnist["dataset"][0][0][1][0][0][0].astype(np.float32)
y_test = emnist["dataset"][0][0][1][0][0][1]

In [5]:
#Normalizing since 255 is the max value
x_train = x_train/x_train.max()
x_test = x_test/x_train.max()

In [6]:
x_train.shape

(240000, 784)

In [7]:
#Flattening pixels into vectors using order A
x_train = x_train.reshape(x_train.shape[0], 1, 28, 28, order = 'A').astype('float32')
x_test = x_test.reshape(x_test.shape[0], 1, 28, 28, order = 'A').astype('float32')

In [8]:
#x_train.shape

In [9]:
#One-hot encode the labels
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)

In [10]:
num_classes = y_test.shape[1]

# Model Function

In [15]:
def create_model(layers, dropout):
    if layers == 1:
        model = Sequential()
        model.add(Conv2D(20, (5, 5), input_shape=(1, 28, 28), activation='relu'))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(dropout))
        model.add(Flatten())
        model.add(Dense(128, activation='relu'))
        model.add(Dense(50, activation='relu'))
        model.add(Dense(num_classes, activation='softmax'))
        # Compile model
        model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
        return model
    if layers == 2:
        model = Sequential()
        model.add(Conv2D(20, (5, 5), input_shape=(1, 28, 28), activation='relu'))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Conv2D(20, (3, 3), activation='relu'))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(dropout))
        model.add(Flatten())
        model.add(Dense(128, activation='relu'))
        model.add(Dense(50, activation='relu'))
        model.add(Dense(num_classes, activation='softmax'))
        # Compile model
        model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
        return model
    if layers == 3:
        model = Sequential()
        model.add(Conv2D(20, (5, 5), input_shape=(1, 28, 28), activation='relu'))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Conv2D(20, (3, 3), activation='relu'))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Conv2D(20, (2, 2), activation='relu'))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(dropout))
        model.add(Flatten())
        model.add(Dense(128, activation='relu'))
        model.add(Dense(50, activation='relu'))
        model.add(Dense(num_classes, activation='softmax'))
        # Compile model
        model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
        return model
    if layers == 4:
        model = Sequential()
        model.add(Conv2D(20, (5, 5), input_shape=(1, 28, 28), activation='relu'))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Conv2D(20, (3, 3), activation='relu'))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Conv2D(20, (2, 2), activation='relu'))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Conv2D(20, (1, 1), activation='relu'))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(dropout))
        model.add(Flatten())
        model.add(Dense(128, activation='relu'))
        model.add(Dense(50, activation='relu'))
        model.add(Dense(num_classes, activation='softmax'))
        # Compile model
        model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
        return model

In [16]:
#model = create_model(1)
#model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=1, batch_size=20000)
#scores = model.evaluate(x_test, y_test, verbose=0)
#print("Large CNN Error: %.2f%%" % (100-scores[1]*100))

In [33]:
#Parameter Tuning
epochs= [5,10,15]
batch_size= [100,200,300]
dropouts = [.2, .5, .75]
#layers=[1,2,3,4]
history=History()

score=[]
data_dict={}
key= 0 
layer=3
#for layer in layers:
for drops in dropouts:
    for epoch in epochs:
        for batch in batch_size:
            model = create_model(layer, drops)
            model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=epoch, batch_size=batch, callbacks=[history])
            scores = model.evaluate(x_test, y_test, verbose=0)
            print("Large CNN Error: %.2f%%" % (100-scores[1]*100))
            print(epoch, batch, drops)

            #Save Iterations in Dictionary
            data_dict[key]={'layer':layer, 'epoch':epoch, 'batch_size':batch, 'dropout_rate':drops}
            score.append(scores[1])
            key += 1  #update Dictionary key

            # summarize history for accuracy
            plt.plot(history.history['acc'])
            plt.plot(history.history['val_acc'])
            plt.title('model accuracy')
            plt.ylabel('accuracy')
            plt.xlabel('epoch')
            plt.legend(['train', 'test'], loc='upper left')
            plt.savefig('../results/plots/acc'+ '%s-%s-%s-%s' % (layer, epoch, batch, int(drops*100))+'.png')
            plt.clf()


            # summarize history for loss
            plt.plot(history.history['loss'])
            plt.plot(history.history['val_loss'])
            plt.title('model loss')
            plt.ylabel('loss')
            plt.xlabel('epoch')
            plt.legend(['train', 'test'], loc='upper left')
            plt.savefig('../results/plots/loss'+ '%s-%s-%s-%s' % (layer, epoch, batch, int(drops*100))+'.png')
            plt.clf()

            data=np.concatenate([history.history['val_loss'],history.history['val_acc'],history.history['loss'],history.history['acc']])
            np.savez_compressed('../results/data/'+ '%s-%s-%s-%s' % (layer, epoch, batch, drops), data)

data= list(zip(score,data_dict.values()))
with open('../results/data.json','w') as outfile:
    json.dump(data,outfile)

Train on 240000 samples, validate on 40000 samples
Epoch 1/5
240000/240000 [==============================] - 211s 880us/step - loss: 0.1712 - acc: 0.9452 - val_loss: 0.2655 - val_acc: 0.9833
Epoch 2/5
240000/240000 [==============================] - 210s 875us/step - loss: 0.0628 - acc: 0.9807 - val_loss: 0.2040 - val_acc: 0.9871
Epoch 3/5
240000/240000 [==============================] - 210s 876us/step - loss: 0.0487 - acc: 0.9850 - val_loss: 0.4314 - val_acc: 0.9724
Epoch 4/5
240000/240000 [==============================] - 211s 878us/step - loss: 0.0418 - acc: 0.9875 - val_loss: 0.2849 - val_acc: 0.9818
Epoch 5/5
240000/240000 [==============================] - 209s 871us/step - loss: 0.0366 - acc: 0.9888 - val_loss: 0.6969 - val_acc: 0.9559
Large CNN Error: 4.41%
5 100 0.2
Train on 240000 samples, validate on 40000 samples
Epoch 1/5
240000/240000 [==============================] - 207s 863us/step - loss: 0.2198 - acc: 0.9304 - val_loss: 0.2534 - val_acc: 0.9840
Epoch 2/5
240000/24

KeyboardInterrupt: 